In [ ]:
Public Const query_Transactions = "ATM Transactions Query"
Public Const sheet_TempTransactions = "ATM Transactions Query"
Public Const sheet_Transactions = "ATM Cash Transactions"
Public Const workbook_Transactions = "ATM_Transactions.xlsx"
Public Const database_ATM = "ATM Database.accdb"
'"shell" subroutine designed to check if the ATM field is blank, then call the necessary subroutines
'to extract the new data, update the table, and then update the cash replenishment schedule
Sub UpdateNewData()
Dim ATM As String
ATM = Range("Selected_ATM").Value
    If ATM = "" Then
        MsgBox ("Please Select an ATM!")
        Exit Sub
    End If
    

    If GetATMTransaction(ATM) Then
        MsgBox ("Successfully processed query")
        'If ImportATMTransactions Then
         '   MsgBox ("Importing of ATM transactions successful")
        End If
    Call ImportATMTransactions
    'End If
End Sub
'this model extracts the ATM transactions from the Access database'
Function GetATMTransaction(ATM As String) As Integer
On Error GoTo GetATMTransaction_Err

    Dim appAccess As Object
    Dim folderPath, selected_ATM, query_string, myTempWorkbookName, myDatabaseFullName, myQuery, myWorkbookName, myTempWorkbookFullName As String
    Dim start_date As Long
    Dim end_date As Long
    
    folderPath = Application.ActiveWorkbook.Path
    myWorkbookName = Application.ActiveWorkbook.Name
    myTempWorkbookName = workbook_Transactions
    myTempWorkbookFullName = folderPath & "\" & myTempWorkbookName
    myDatabaseFullName = folderPath & "\" & database_ATM
    myQuery = query_Transactions
    start_date = Range("Selected_Start_Date")
    end_date = Range("Selected_End_Date") + 1
    
    selected_ATM = Range("Selected_ATM").Value
    query_string = "[ATM Transactions].[ATM ID] ='" & selected_ATM & "' And [ATM Transactions].[Timestamp] >= " & start_date & " And [ATM Transactions].[Timestamp] < " & end_date
             
    Set appAccess = CreateObject("Access.Application")
    appAccess.OpenCurrentDatabase myDatabaseFullName
    appAccess.DoCmd.OpenQuery myQuery, acViewNormal, acReadOnly
    appAccess.DoCmd.ApplyFilter , query_string, ""
    appAccess.DoCmd.OutputTo acOutputQuery, myQuery, "ExcelWorkbook(*.xlsx)", myTempWorkbookFullName, False
    appAccess.Quit
    
    
    
GetATMTransaction_Exit:
    GetATMTransaction = 1
    Exit Function
    
GetATMTransaction_Err:
    appAccess.Quit
    MsgBox ("ERROR in executing the query.")
    GetATMTransaction = 0
    Exit Function
    
End Function

'this subroutine is designed to update the cash transactions tab in the tool'
Function ImportATMTransactions()
On Error GoTo ImportATMTransactions_Err
Dim folderPath, myWorkbookName, myWorksheetName, myTempWorkbookName, myTempWorkbookFullName, myTempWorksheetName, newAddress, transactionData As String
Dim numRows, numCol, numRowNew, numRowCol As Integer
Dim timestamp As String
Dim ATMid, account, transactionType  As String
Dim amount, transactionFee As String
    
    timestamp = "Timestamp"
    ATMid = "ATM_id"
    account = "Account"
    transactionType = "Transaction_Type"
    amount = "Amount"
    transactionFee = "Transaction_Fee"
    
    transactionData = "Transaction_Data"
    numCol = Range(transactionData).Columns.Count
    numRows = Range(transactionData).Rows.Count
    folderPath = Application.ActiveWorkbook.Path
    myWorkbookName = Application.ActiveWorkbook.Name
    myWorksheetName = sheet_Transactions
    
    myTempWorkbookName = workbook_Transactions
    myTempWorkbookFullName = folderPath & "\" & myTempWorkbookName
    myTempWorksheetName = sheet_TempTransactions
    
    Application.ScreenUpdating = False
    Application.Calculation = xlManual
    
    Workbooks.Open (myTempWorkbookFullName)
    Sheets(myTempWorksheetName).Select
    If (Range("A2") = "") Then
        Workbooks(myTempWorkbookName).Close
        MsgBox ("ERROR! No records were received.")
        Exit Function
    End If
    
    'Clear Existing Data
    Workbooks(myWorkbookName).Activate
    Sheets(myWorksheetName).Select
    Range(transactionData).Select
    Selection.ClearContents
    Range(transactionData).Cells(2, 1).Select
    If numRows > 1 Then
        Range(Selection, Selection.Offset(numRows - 2, numCol - 1)).Clear
        End If
        
    
    'Repopulate with imported data
    Workbooks(myTempWorkbookName).Activate
    Sheets(myTempWorksheetName).Select
    Range("A2").Select
    Range(Selection, Selection.End(xlToRight).End(xlDown)).Select
    numRowNew = Selection.Rows.Count
    numColNew = Selection.Columns.Count
    Selection.Copy
    Workbooks(myWorkbookName).Activate
    Sheets(myWorksheetName).Select
    Range(transactionData).Cells(1, 1).Select
    Selection.PasteSpecial Paste:=xlPasteValues, Operation:=xlNone, SkipBlanks _
        :=False, Transpose:=False
        
    'rename the section
    
        
Range(timestamp).Cells(1, 1).Select
    newAddress = Range(Selection, Selection.Offset(numRowNew - 1, 0)).address
        
    
    With ActiveWorkbook.Names(timestamp)
           .RefersTo = "='" & myWorksheetName & "'!" & newAddress
   End With
    
Range(ATMid).Cells(1, 1).Select
    newAddress = Range(Selection, Selection.Offset(numRowNew - 1, 0)).address
    
    With ActiveWorkbook.Names(ATMid)
            .RefersTo = "='" & myWorksheetName & "'!" & newAddress
    End With
            
Range(account).Cells(1, 1).Select
    newAddress = Range(Selection, Selection.Offset(numRowNew - 1, 0)).address
        
   
    With ActiveWorkbook.Names(account)
            .RefersTo = "='" & myWorksheetName & "'!" & newAddress
    End With

 Range(amount).Cells(1, 1).Select
    newAddress = Range(Selection, Selection.Offset(numRowNew - 1, 0)).address
   
    
    With ActiveWorkbook.Names(amount)
            .RefersTo = "='" & myWorksheetName & "'!" & newAddress
    End With
    
 Range(transactionType).Cells(1, 1).Select
    newAddress = Range(Selection, Selection.Offset(numRowNew - 1, 0)).address
        
   
    
    With ActiveWorkbook.Names(transactionType)
            .RefersTo = "='" & myWorksheetName & "'!" & newAddress
    End With
    
 Range(transactionFee).Cells(1, 1).Select
    newAddress = Range(Selection, Selection.Offset(numRowNew - 1, 0)).address
   
    
    With ActiveWorkbook.Names(transactionFee)
            .RefersTo = "='" & myWorksheetName & "'!" & newAddress
    End With
    
Range(transactionData).Cells(1, 1).Select
        newAddress = Range(Selection, Selection.Offset(numRowNew - 1, numColNew - 1)).address
        With ActiveWorkbook.Names(transactionData)
                .RefersTo = "='" & myWorksheetName & "'!" & newAddress
        End With
        
    newAddress = Range(transactionData).address
        If (Range(newAddress).Rows.Count > 1) Then
            Range(transactionData).Rows(1).Copy
            Range(newAddress).PasteSpecial Paste:=xlPasteFormats, Operation:=xlNone, SkipBlanks:=False, Transpose:=False
            Application.CutCopyMode = False
        
        End If
        
    Application.CutCopyMode = False
    Workbooks(myTempWorkbookName).Close
   
    Call UpdateYellowColumns
    
    Application.ScreenUpdating = True
    Application.Calculation = xlAutomatic
    
ImportATMTransactions_Exit:
    ImportATMTransactions = 1
    Exit Function
    
ImportATMTransactions_Err:
    MsgBox ("ERROR in executing the import.")
    appAccess.Quit
    ImportATMTransactions = 0
    Exit Function
End Function

Sub UpdateYellowColumns()
 Dim timeSince, cashBefore, withdrawn, fees, cashAfter, failed, lostfees, day, dayWeek, dayNum As String
 Dim timeSinceFormula, cashBeforeFormula, withdrawnFormula, feesFormula, cashAfterFormula, failedFormula, lostfeesFormula, dayFormula, dayWeekFormula, dayNumFormula As String
 Dim numRow As Integer
 Dim worksheetName As String
 Dim newAddress As String
 
 timeSince = "Time_Since_Last_Transaction"
 cashBefore = "ATM_Cash_Before_Transaction"
 withdrawn = "Amount_Withdrawn"
 fees = "Transaction_Fees_Collected"
 cashAfter = "ATM_Cash_After_Transaction"
 failed = "Failed_Transactions"
 lostfees = "Lost_Transaction_Fees"
 day = "Day"
 dayWeek = "Day_of_Week"
 dayNum = "Day_Num"
 worksheetName = sheet_Transactions
 
 timeSinceFormula = Range(timeSince).Cells(1, 1).Formula
 cashBeforeFIRSTFormula = Range(cashBefore).Cells(1, 1).Formula
 cashBeforeTRUEFormula = Range(cashBefore).Cells(2, 1).Formula
 withdrawnFormula = Range(withdrawn).Cells(1, 1).Formula
 feesFormula = Range(fees).Cells(1, 1).Formula
 cashAfterFormula = Range(cashAfter).Cells(1, 1).Formula
 failedFormula = Range(failed).Cells(1, 1).Formula
 lostfeesFormula = Range(lostfees).Cells(1, 1).Formula
 dayFormula = Range(day).Cells(1, 1).Formula
 dayWeekFormula = Range(dayWeek).Cells(1, 1).Formula
 dayNumFormula = Range(dayNum).Cells(1, 1).Formula
 
 tableToUpdate = "Yellow_Table"
 NumColCurrent = Range(tableToUpdate).Columns.Count
 numRowCurrent = Range(tableToUpdate).Rows.Count
 numRowFuture = Range("Transaction_Data").Rows.Count
 
 Range(tableToUpdate).ClearContents
 Range(tableToUpdate).Cells(2, 1).Select
    If numRowCurrent > 1 Then
        Range(Selection, Selection.Offset(numRowCurrent - 2, NumColCurrent - 1)).Clear
        End If

Range(tableToUpdate).Cells(1, 1).Select
    newAddress = Range(Selection, Selection.Offset(numRowFuture - 1, NumColCurrent - 1)).address
    With ActiveWorkbook.Names(tableToUpdate)
            .RefersTo = "='" & worksheetName & "'!" & newAddress
    End With
    
Range(timeSince).Cells(1, 1).Select
    newAddress = Range(Selection, Selection.Offset(numRowFuture - 1, 0)).address
    Selection.Formula = timeSinceFormula
        
    If (Range(newAddress).Rows.Count > 1) Then
        Selection.AutoFill Destination:=Range(newAddress)
    End If
    
    With ActiveWorkbook.Names(timeSince)
            .RefersTo = "='" & worksheetName & "'!" & newAddress
    End With
 
 Range(cashBefore).Cells(1, 1).Select
    Selection.Formula = cashBeforeFIRSTFormula
    Range(cashBefore).Cells(2, 1).Select
    newAddress = Range(Selection, Selection.Offset(numRowFuture - 2, 0)).address
    Selection.Formula = cashBeforeTRUEFormula
        
    If (Range(newAddress).Rows.Count > 1) Then
        Selection.AutoFill Destination:=Range(newAddress)
    End If
    
    Range(cashBefore).Cells(1, 1).Select
    newAddress = Range(Selection, Selection.Offset(numRowFuture - 1, 0)).address
    
    With ActiveWorkbook.Names(cashBefore)
            .RefersTo = "='" & worksheetName & "'!" & newAddress
    End With
 
 Range(withdrawn).Cells(1, 1).Select
    newAddress = Range(Selection, Selection.Offset(numRowFuture - 1, 0)).address
    Selection.Formula = withdrawnFormula
        
    If (Range(newAddress).Rows.Count > 1) Then
        Selection.AutoFill Destination:=Range(newAddress)
    End If
    
    With ActiveWorkbook.Names(withdrawn)
            .RefersTo = "='" & worksheetName & "'!" & newAddress
    End With
 
 Range(fees).Cells(1, 1).Select
    newAddress = Range(Selection, Selection.Offset(numRowFuture - 1, 0)).address
    Selection.Formula = feesFormula
        
    If (Range(newAddress).Rows.Count > 1) Then
        Selection.AutoFill Destination:=Range(newAddress)
    End If
    
    With ActiveWorkbook.Names(fees)
            .RefersTo = "='" & worksheetName & "'!" & newAddress
    End With
 
 Range(cashAfter).Cells(1, 1).Select
    newAddress = Range(Selection, Selection.Offset(numRowFuture - 1, 0)).address
    Selection.Formula = cashAfterFormula
        
    If (Range(newAddress).Rows.Count > 1) Then
        Selection.AutoFill Destination:=Range(newAddress)
    End If
    
    With ActiveWorkbook.Names(cashAfter)
            .RefersTo = "='" & worksheetName & "'!" & newAddress
    End With
 
 Range(failed).Cells(1, 1).Select
    newAddress = Range(Selection, Selection.Offset(numRowFuture - 1, 0)).address
    Selection.Formula = failedFormula
        
    If (Range(newAddress).Rows.Count > 1) Then
        Selection.AutoFill Destination:=Range(newAddress)
    End If
    
    With ActiveWorkbook.Names(failed)
            .RefersTo = "='" & worksheetName & "'!" & newAddress
    End With
 
 Range(lostfees).Cells(1, 1).Select
    newAddress = Range(Selection, Selection.Offset(numRowFuture - 1, 0)).address
    Selection.Formula = lostfeesFormula
        
    If (Range(newAddress).Rows.Count > 1) Then
        Selection.AutoFill Destination:=Range(newAddress)
    End If
    
    With ActiveWorkbook.Names(lostfees)
            .RefersTo = "='" & worksheetName & "'!" & newAddress
    End With
    
'day
Range(day).Cells(1, 1).Select
    newAddress = Range(Selection, Selection.Offset(numRowFuture - 1, 0)).address
    Selection.Formula = dayFormula
        
    If (Range(newAddress).Rows.Count > 1) Then
        Selection.AutoFill Destination:=Range(newAddress)
    End If
    
    With ActiveWorkbook.Names(day)
            .RefersTo = "='" & worksheetName & "'!" & newAddress
    End With
    
'dayWeek
Range(dayWeek).Cells(1, 1).Select
    newAddress = Range(Selection, Selection.Offset(numRowFuture - 1, 0)).address
    Selection.Formula = dayWeekFormula
        
    If (Range(newAddress).Rows.Count > 1) Then
        Selection.AutoFill Destination:=Range(newAddress)
    End If
    
    With ActiveWorkbook.Names(dayWeek)
            .RefersTo = "='" & worksheetName & "'!" & newAddress
    End With
    
'dayNum
Range(dayNum).Cells(1, 1).Select
    newAddress = Range(Selection, Selection.Offset(numRowFuture - 1, 0)).address
    Selection.Formula = dayNumFormula
        
    If (Range(newAddress).Rows.Count > 1) Then
        Selection.AutoFill Destination:=Range(newAddress)
    End If
    
    With ActiveWorkbook.Names(dayNum)
            .RefersTo = "='" & worksheetName & "'!" & newAddress
    End With
 
        
    newAddress = Range(tableToUpdate).address
    
    If (Range(newAddress).Rows.Count > 1) Then
        Range(tableToUpdate).Rows(1).Copy
        Range(newAddress).PasteSpecial Paste:=xlPasteFormats, Operation:=xlNone, _
            SkipBlanks:=False, Transpose:=False
        Application.CutCopyMode = False
    End If

Application.ScreenUpdating = True
Application.Calculation = xlAutomatic
    
End Sub


Function QueryATMOffPremise() As Integer
On Error GoTo QueryATMOffPremise_Err
Dim appAccess As Object
Dim folderPath, status, query_string, myTempWorkbookName, myDatabaseFullName, myQuery, myWorkbookName, myTempWorkbookFullName As String
    
    folderPath = Application.ActiveWorkbook.Path
    myTempWorkbookName = "ATM_Status.xlsx"
    myTempWorkbookFullName = folderPath & "\" & myTempWorkbookName
    myDatabaseFullName = folderPath & "\" & database_ATM
    myQuery = "ATM Query"
    
    status = "OFF"
    query_string = "[ATM].[Type] ='" & status & "'"
    
    Set appAccess = CreateObject("Access.Application")
    appAccess.OpenCurrentDatabase myDatabaseFullName
    appAccess.DoCmd.OpenQuery myQuery, acViewNormal, acReadOnly
    appAccess.DoCmd.ApplyFilter , query_string, ""
    appAccess.DoCmd.OutputTo acOutputQuery, myQuery, "ExcelWorkbook(*.xlsx)", myTempWorkbookFullName, False
    appAccess.Quit
    
QueryATMOffPremise_Exit:
        QueryATMOffPremise = 1
        Exit Function
    
QueryATMOffPremise_Err:
        appAccess.Quit
        MsgBox ("Error executing the ATM query")
        QueryATMOffPremise = 0
        Exit Function
        
End Function
Function ImportATMOffPremise() As Integer
On Error GoTo ImportATMOffPremise_Err
Dim folderPath, myWorkbookName, myWorksheetName, myTempWorkbookName, myTempWorkbookFullName, myTempWorksheetName, newAddress, typeData As String
Dim numCol, numRow, numRowNew As Integer

    typeData = "Off_Premise_ATM_List"
    numCol = Range(typeData).Columns.Count
    numRow = Range(typeData).Rows.Count
    folderPath = Application.ActiveWorkbook.Path
    myWorkbookName = Application.ActiveWorkbook.Name
    myWorksheetName = "Off-Premise ATM List"
    
    myTempWorkbookName = "ATM_Status.xlsx"
    myTempWorkbookFullName = folderPath & "\" & myTempWorkbookName
    myTempWorksheetName = "ATM Query"
    
    Application.ScreenUpdating = False
    Application.Calculation = xlManual
    
    Workbooks.Open (myTempWorkbookFullName)
    Sheets(myTempWorksheetName).Select
    If (Range("A2") = "") Then
        Workbooks(myTempWorkbookName).Close
        MsgBox ("Error! No Off Premise ATMs")
        Exit Function
    End If
    
    Workbooks(myWorkbookName).Activate
    Sheets(myWorksheetName).Select
    Range(typeData).Select
    Selection.ClearContents
    Range(typeData).Cells(2, 1).Select
    If numRow > 1 Then
        Range(Selection, Selection.Offset(numRow - 1, 0)).Clear
    End If
    
    Workbooks(myTempWorkbookName).Activate
    Sheets(myTempWorksheetName).Select
    Range("A2").Select
    Range(Selection, Selection.End(xlDown)).Select
    numRowNew = Selection.Rows.Count
    Selection.Copy
    Workbooks(myWorkbookName).Activate
    Sheets(myWorksheetName).Activate
    Sheets(myWorksheetName).Select
    Range(typeData).Cells(1, 1).Select
    Selection.PasteSpecial Paste:=xlPasteValues, Operation:=xlNone, SkipBlanks:=False, Transpose:=False
    
    Range(typeData).Cells(1, 1).Select
        newAddress = Range(Selection, Selection.Offset(numRowNew - 1, 0)).address
        With ActiveWorkbook.Names(typeData)
                .RefersTo = "='" & myWorksheetName & "'!" & newAddress
        End With
        
    newAddress = Range(typeData).address
        If (Range(newAddress).Rows.Count > 1) Then
            Range(typeData).Rows(1).Copy
            Range(newAddress).PasteSpecial Paste:=xlPasteFormats, Operation:=xlNone, SkipBlanks:=False, Transpose:=False
            Application.CutCopyMode = False
        End If
        
     Application.CutCopyMode = False
     Workbooks(myTempWorkbookName).Close
     Application.ScreenUpdating = True
     Application.Calculation = xlAutomatic
     
ImportATMOffPremise_Exit:
     ImportATMOffPremise = 1
     Exit Function
     
ImportATMOffPremise_Err:
     MsgBox ("Error in importing off premise ATMs")
     appAccess.Quit
     ImportATMOffPremise = 0
     Exit Function
    
End Function
        

Sub Test()
    If QueryATMOffPremise Then
        MsgBox ("Yes")
        If ImportATMOffPremise Then
            MsgBox ("Fuck Yes!")
        Else
            MsgBox ("ugh")
        End If
    Else
        MsgBox ("No")
        
    End If
    
End Sub

Sub schedule_opt()


Dim period_length As Integer
Dim day(1 To 7) As Integer
Dim avg_daily_demand(1 To 7) As Integer
Dim Mon As Integer
Dim Tues As Integer
Dim Wed As Integer
Dim Thur As Integer
Dim Fri As Integer
Dim Sat As Integer
Dim Sun As Integer
Dim start_date As Long
Dim Num_of_Rep As Integer
Dim Rep_Amount As Integer
Dim days_of_week(1 To 7) As String
Dim temp_max As Integer
Dim temp_max_index As Integer
Dim rep_amount_1 As Integer
Dim rep_amount_2 As Integer
'Dim amount As Integer

'amount = Range("Amount")


Application.ScreenUpdating = False
Application.Calculation = xlManual

period_length = Range("End_Date").Value - Range("Start_Date").Value + 1

days_of_week(1) = "Sunday"
days_of_week(2) = "Monday"
days_of_week(3) = "Tuesday"
days_of_week(4) = "Wednesday"
days_of_week(5) = "Thursday"
days_of_week(6) = "Friday"
days_of_week(7) = "Saturday"

Sun = 0
Mon = 0
Tues = 0
Wed = 0
Thur = 0
Fri = 0
Sat = 0

'sums daily demand
Sun_Demand = Application.WorksheetFunction.SumIf(Range("Day_Num"), 1, Range("Amount"))
'Sun_Demand = Application.WorksheetFunction.SumIfs(Range("Amount"), Range("Day_Num"), 1)
Mon_Demand = Application.WorksheetFunction.SumIf(Range("Day_Num"), 2, Range("Amount"))
Tues_Demand = Application.WorksheetFunction.SumIf(Range("Day_Num"), 3, Range("Amount"))
Wed_Demand = Application.WorksheetFunction.SumIf(Range("Day_Num"), 4, Range("Amount"))
Thur_Demand = Application.WorksheetFunction.SumIf(Range("Day_Num"), 5, Range("Amount"))
Fri_Demand = Application.WorksheetFunction.SumIf(Range("Day_Num"), 6, Range("Amount"))
Sat_Demand = Application.WorksheetFunction.SumIf(Range("Day_Num"), 7, Range("Amount"))


'calculates number of each individual day
    For j = 1 To Range("Timestamp").Rows.Count
    
        If Range("Day_Num").Cells(j, 1).Value <> Range("Day_Num").Cells(j - 1, 1).Value Then
            
            If Range("Day_Num").Cells(j, 1).Value = 1 Then
                Sun = Sun + 1
            End If
            If Range("Day_Num").Cells(j, 1).Value = 2 Then
                Mon = Mon + 1
            End If
            If Range("Day_Num").Cells(j, 1).Value = 3 Then
                Tues = Tues + 1
            End If
            If Range("Day_Num").Cells(j, 1).Value = 4 Then
                Wed = Wed + 1
            End If
            If Range("Day_Num").Cells(j, 1).Value = 5 Then
                Thur = Thur + 1
            End If
            If Range("Day_Num").Cells(j, 1).Value = 6 Then
                Fri = Fri + 1
            End If
            If Range("Day_Num").Cells(j, 1).Value = 7 Then
                Sat = Sat + 1
            End If
            
        End If

    Next j
        
  'produces average DAILY demand for each day
  For i = 1 To 7
    
    If i = 1 And Sun <> 0 Then
    avg_daily_demand(i) = Sun_Demand / Sun
    End If
    
    If i = 2 And Mon <> 0 Then
    avg_daily_demand(i) = Mon_Demand / Mon
    End If
    
    If i = 3 And Tues <> 0 Then
    avg_daily_demand(i) = Tues_Demand / Tues
    End If
    
    If i = 4 And Wed <> 0 Then
    avg_daily_demand(i) = Wed_Demand / Wed
    End If
    If i = 5 And Thur <> 0 Then
    avg_daily_demand(i) = Thur_Demand / Thur
    End If
    
    If i = 6 And Fri <> 0 Then
    avg_daily_demand(i) = Fri_Demand / Fri
    End If
    
    If i = 7 And Sat <> 0 Then
    avg_daily_demand(i) = Sat_Demand / Sat
    End If

Next i

'average total WEEKLY demand
total_weekly = Application.WorksheetFunction.Sum(avg_daily_demand(1), avg_daily_demand(2), avg_daily_demand(3), avg_daily_demand(4), avg_daily_demand(5), avg_daily_demand(5), avg_daily_demand(6), avg_daily_demand(7))
Num_of_Rep = Application.WorksheetFunction.Ceiling(total_weekly / 20000, 1)

'Num_of_Rep = 4
'iterating through all num_rep combos

'x is the max daily demand
x = Application.WorksheetFunction.Max(avg_daily_demand)
'y is the max daily demand index
y = Application.WorksheetFunction.Match(x, avg_daily_demand, 0)
'min daily demand
Z = Application.WorksheetFunction.Min(avg_daily_demand)
'zz is the min daily demand index
ZZ = Application.WorksheetFunction.Match(Z, avg_daily_demand, 0)

'checks cases of how many replenishments per week and produces weekly schedule
If Num_of_Rep = 1 Then
    
   Rep_Amount = total_weekly - (total_weekly Mod 20)
   temp_max = -9999
   temp_max_index = 0
   
    For k = 1 To 7
            
            Range("Selected_Days").ClearContents
            Range("R_Amounts").Cells(k, 1).Value = Rep_Amount
            Range("R_Times").Cells(k, 1).Value = "4:00:00 AM"
            Call Day_Toggle(days_of_week(k))
             
                If Range("FCI").Cells(1, 1).Value < Range("FCI_limit").Cells(1, 1).Value Then
                    
                    If Range("Profit").Cells(1, 1).Value > temp_max Then
                        temp_max = Range("Profit").Cells(1, 1).Value
                        temp_max_index = k
                    End If
                End If
    Next k
    'If a solution was not found then another try is made with a higher replenishment value
    If temp_max = -9999 Then
        For k = 1 To 7
            
            Range("Selected_Days").ClearContents
            Range("R_Amounts").Cells(k, 1).Value = Application.WorksheetFunction.Min(20000, Rep_Amount * 1.25)
            Call Day_Toggle(days_of_week(k))
             
                If Range("FCI").Cells(1, 1).Value < Range("FCI_limit").Cells(1, 1).Value Then
                    
                    If Range("Profit").Cells(1, 1).Value > temp_max Then
                        temp_max = Range("Profit").Cells(1, 1).Value
                        temp_max_index = k
                    End If
                End If
        Next k
    End If
    
    'checks to make sure a feasible schedule has been found, adds a replenishment if not
    If temp_max = -9999 Then
        Num_Rep = 2
    Else
    
    Range("Selected_Days").ClearContents
        Call Day_Toggle(days_of_week(temp_max_index))
    End If
    
End If

'Start of replenishment 2 case
'This case always replenishes on the day with the highest daily demand to reduce the size of the solution set, decreasing run time
    If Num_of_Rep = 2 Then
        Rep_Amount = Application.WorksheetFunction.Min(CInt(total_weekly / 1.6 - (total_weekly / 1.6 Mod 20)), Range("ATM_Cash_Limit").Cells(1, 1).Value)
        'Rep_Amount = CInt(total_weekly / 1.5 - (total_weekly / 1.5 Mod 20))
        
        temp_max = -9999
        temp_max_index = 0
        
        For q = 1 To 7
            
            Range("Selected_Days").ClearContents
            Range("R_Times").Cells(q, 1).Value = "4:00:00 AM"
            
            If q <> y And q <> y - 1 And q <> y + 1 And q <> y - 6 And q <> y + 6 Then
            
                Range("R_Amounts").Cells(q, 1).Value = Rep_Amount
                Range("R_Amounts").Cells(y, 1).Value = Rep_Amount
                Call Day_Toggle(days_of_week(y))
                Call Day_Toggle(days_of_week(q))
                
                If Range("FCI").Cells(1, 1).Value < Range("FCI_limit").Cells(1, 1).Value Then
                    
                    If Range("Profit").Cells(1, 1).Value > temp_max Then
                        temp_max = Range("Profit").Cells(1, 1).Value
                        temp_max_index = q
                    End If
                End If
            End If
        Next q
        'If a solution was not found then another try is made with a higher replenishment value
        If temp_max = -9999 Then
            For q = 1 To 7
            
            Range("Selected_Days").ClearContents
            
            If q <> y And q <> y - 1 And q <> y + 1 And q <> y - 6 And q <> y + 6 Then
            
                Range("R_Amounts").Cells(q, 1).Value = Application.WorksheetFunction.Min(20000, Rep_Amount * 1.35 - (Rep_Amount * 1.35 Mod 20))
                Range("R_Amounts").Cells(y, 1).Value = Application.WorksheetFunction.Min(20000, Rep_Amount * 1.35 - (Rep_Amount * 1.35 Mod 20))
                Call Day_Toggle(days_of_week(y))
                Call Day_Toggle(days_of_week(q))
                
                If Range("FCI").Cells(1, 1).Value < Range("FCI_limit").Cells(1, 1).Value Then
                    
                    If Range("Profit").Cells(1, 1).Value > temp_max Then
                        temp_max = Range("Profit").Cells(1, 1).Value
                        temp_max_index = q
                    End If
                End If
            End If
            Next q
        End If
        'checks to make sure a feasible schedule has been found, adds a replenishment if not
        If temp_max = -9999 Then
            Num_Rep = 3
        Else
        
            Range("Selected_Days").ClearContents
            Call Day_Toggle(days_of_week(temp_max_index))
            Call Day_Toggle(days_of_week(y))
    
        End If

    End If

'Start of 3 replenishment case

If Num_of_Rep = 3 Then
    Rep_Amount = Application.WorksheetFunction.Min(CInt(total_weekly / 2.5 - (total_weekly / 2.5 Mod 20)), Range("ATM_Cash_Limit").Cells(1, 1).Value)
    temp_max = -9999
    temp_max_index = 0
    
For p = 1 To 7
    Range("R_Times").Cells(p, 1).Value = "4:00:00 AM"
    If p <> y Then
    
    
     For s = 1 To 7
            
            Range("Selected_Days").ClearContents
            
            
            If s <> y And s <> p Then
            
                Range("R_Amounts").Cells(s, 1).Value = Rep_Amount
                Range("R_Amounts").Cells(y, 1).Value = Rep_Amount
                Range("R_Amounts").Cells(p, 1).Value = Rep_Amount
                Call Day_Toggle(days_of_week(y))
                Call Day_Toggle(days_of_week(s))
                Call Day_Toggle(days_of_week(p))
                
                If Range("FCI").Cells(1, 1).Value < Range("FCI_limit").Cells(1, 1).Value Then
                    
                    If Range("Profit").Cells(1, 1).Value > temp_max Then
                        temp_max = Range("Profit").Cells(1, 1).Value
                        temp_max_index1 = s
                        temp_max_index2 = p
                    End If
                End If
            End If
    Next s
        
    End If
    
Next p
'If a solution was not found then another try is made with a higher replenishment value
If temp_max = -9999 Then
    For p = 1 To 7

    If p <> y Then
    
    
     For s = 1 To 7
            
            Range("Selected_Days").ClearContents
            
            If s <> y And s <> p Then
            
                Range("R_Amounts").Cells(s, 1).Value = Application.WorksheetFunction.Min(20000, Rep_Amount * 1.5 - (Rep_Amount * 1.5 Mod 20))
                Range("R_Amounts").Cells(y, 1).Value = Application.WorksheetFunction.Min(20000, Rep_Amount * 1.5 - (Rep_Amount * 1.5 Mod 20))
                Range("R_Amounts").Cells(p, 1).Value = Application.WorksheetFunction.Min(20000, Rep_Amount * 1.5 - (Rep_Amount * 1.5 Mod 20))
                Call Day_Toggle(days_of_week(y))
                Call Day_Toggle(days_of_week(s))
                Call Day_Toggle(days_of_week(p))
                
                If Range("FCI").Cells(1, 1).Value < Range("FCI_limit").Cells(1, 1).Value Then
                    
                    If Range("Profit").Cells(1, 1).Value > temp_max Then
                        temp_max = Range("Profit").Cells(1, 1).Value
                        temp_max_index1 = s
                        temp_max_index2 = p
                    End If
                End If
            End If
    Next s
        
    End If
    
Next p
End If
    'checks to make sure a feasible schedule has been found, adds a replenishment if not
    If temp_max = -9999 Then
        Num_Rep = 4
    Else
        
        Range("Selected_Days").ClearContents
        Call Day_Toggle(days_of_week(temp_max_index1))
        Call Day_Toggle(days_of_week(temp_max_index2))
        Call Day_Toggle(days_of_week(y))
    
    End If
    

End If


'Start of 4 replenishment case
If Num_of_Rep = 4 Then
    Rep_Amount = Application.WorksheetFunction.Min(CInt(total_weekly / 3.5 - (total_weekly / 3.5 Mod 20)), Range("ATM_Cash_Limit").Cells(1, 1).Value)
    temp_max = -9999
    temp_max_index = 0
    
For p = 1 To 7
    Range("R_Times").Cells(p, 1).Value = "4:00:00 AM"
    If p <> y Then
    
    For d = 1 To 7
        Range("Selected_Days").ClearContents
        If d <> y And d <> p Then
        
        For s = 1 To 7

            If s <> y And s <> p And s <> d Then
            
                Range("R_Amounts").Cells(s, 1).Value = Rep_Amount
                Range("R_Amounts").Cells(y, 1).Value = Rep_Amount
                Range("R_Amounts").Cells(p, 1).Value = Rep_Amount
                Range("R_Amounts").Cells(d, 1).Value = Rep_Amount
                Call Day_Toggle(days_of_week(y))
                Call Day_Toggle(days_of_week(s))
                Call Day_Toggle(days_of_week(p))
                Call Day_Toggle(days_of_week(d))
                
                If Range("FCI").Cells(1, 1).Value < Range("FCI_limit").Cells(1, 1).Value Then
                    
                    If Range("Profit").Cells(1, 1).Value > temp_max Then
                        temp_max = Range("Profit").Cells(1, 1).Value
                        temp_max_index1 = s
                        temp_max_index2 = p
                        temp_max_index3 = d
                    End If
                End If
            End If
        Next s
        End If
    Next d
    End If

Next p
'If a solution was not found then another try is made with a higher replenishment value
If temp_max = -9999 Or temp_max = -5872 Then
    For p = 1 To 7
    Range("R_Times").Cells(p, 1).Value = "4:00:00 AM"
    If p <> y Then
    
    For d = 1 To 7
        Range("Selected_Days").ClearContents
        If d <> y And d <> p Then
        
        For s = 1 To 7

            If s <> y And s <> p And s <> d Then
            
                Range("R_Amounts").Cells(s, 1).Value = Application.WorksheetFunction.Min(20000, Rep_Amount * 1.8 - (Rep_Amount * 1.8 Mod 20))
                Range("R_Amounts").Cells(y, 1).Value = Application.WorksheetFunction.Min(20000, Rep_Amount * 1.8 - (Rep_Amount * 1.8 Mod 20))
                Range("R_Amounts").Cells(p, 1).Value = Application.WorksheetFunction.Min(20000, Rep_Amount * 1.8 - (Rep_Amount * 1.8 Mod 20))
                Range("R_Amounts").Cells(d, 1).Value = Application.WorksheetFunction.Min(20000, Rep_Amount * 1.8 - (Rep_Amount * 1.8 Mod 20))
                Call Day_Toggle(days_of_week(y))
                Call Day_Toggle(days_of_week(s))
                Call Day_Toggle(days_of_week(p))
                Call Day_Toggle(days_of_week(d))
                
                If Range("FCI").Cells(1, 1).Value < Range("FCI_limit").Cells(1, 1).Value Then
                    
                    If Range("Profit").Cells(1, 1).Value > temp_max Then
                        temp_max = Range("Profit").Cells(1, 1).Value
                        temp_max_index1 = s
                        temp_max_index2 = p
                        temp_max_index3 = d
                    End If
                End If
            End If
        Next s
        End If
    Next d
    End If
    
    Next p
End If
If temp_max = -9999 Or temp_max = -5872 Then
    For p = 1 To 7
    Range("R_Times").Cells(p, 1).Value = "4:00:00 AM"
    If p <> y Then
    
    For d = 1 To 7
        Range("Selected_Days").ClearContents
        If d <> y And d <> p Then
        
        For s = 1 To 7

            If s <> y And s <> p And s <> d Then
            
                Range("R_Amounts").Cells(s, 1).Value = Application.WorksheetFunction.Min(20000, Rep_Amount * 2.1 - (Rep_Amount * 2.1 Mod 20))
                Range("R_Amounts").Cells(y, 1).Value = Application.WorksheetFunction.Min(20000, Rep_Amount * 2.1 - (Rep_Amount * 2.1 Mod 20))
                Range("R_Amounts").Cells(p, 1).Value = Application.WorksheetFunction.Min(20000, Rep_Amount * 2.1 - (Rep_Amount * 2.1 Mod 20))
                Range("R_Amounts").Cells(d, 1).Value = Application.WorksheetFunction.Min(20000, Rep_Amount * 2.1 - (Rep_Amount * 2.1 Mod 20))
                Call Day_Toggle(days_of_week(y))
                Call Day_Toggle(days_of_week(s))
                Call Day_Toggle(days_of_week(p))
                Call Day_Toggle(days_of_week(d))
                MsgBox (temp_max)
                If Range("FCI").Cells(1, 1).Value < Range("FCI_limit").Cells(1, 1).Value Then
                    
                    If Range("Profit").Cells(1, 1).Value > temp_max Then
                        temp_max = Range("Profit").Cells(1, 1).Value
                        temp_max_index1 = s
                        temp_max_index2 = p
                        temp_max_index3 = d
                    End If
                End If
            End If
        Next s
        End If
    Next d
    End If
    
    Next p
End If
    'checks to make sure a feasible schedule has been found, adds a replenishment if not
    If temp_max = -9999 Then
        Num_Rep = 5
    Else
        
        Range("Selected_Days").ClearContents
        Call Day_Toggle(days_of_week(temp_max_index1))
        Call Day_Toggle(days_of_week(temp_max_index2))
        Call Day_Toggle(days_of_week(temp_max_index3))
        Call Day_Toggle(days_of_week(y))
    
    End If
    

End If


'Start of 5 replenishment case
If Num_of_Rep = 5 Then
    Rep_Amount = Application.WorksheetFunction.Min(CInt(total_weekly / 4.2 - (total_weekly / 4.2 Mod 20)), Range("ATM_Cash_Limit").Cells(1, 1).Value)
    temp_max = -9999
    temp_max_index = 0
    
For p = 1 To 7
    Range("R_Times").Cells(p, 1).Value = "4:00:00 AM"
    If p <> y Then
    For h = 1 To 7
    
    If h <> p And h <> y Then
    For d = 1 To 7
        Range("Selected_Days").ClearContents
        If d <> y And d <> h And d <> p Then
        
        For s = 1 To 7

            If s <> y And s <> d And s <> p And s <> d Then
            
                Range("R_Amounts").Cells(s, 1).Value = Rep_Amount
                Range("R_Amounts").Cells(y, 1).Value = Rep_Amount
                Range("R_Amounts").Cells(p, 1).Value = Rep_Amount
                Range("R_Amounts").Cells(d, 1).Value = Rep_Amount
                Range("R_Amounts").Cells(h, 1).Value = Rep_Amount
                Call Day_Toggle(days_of_week(y))
                Call Day_Toggle(days_of_week(s))
                Call Day_Toggle(days_of_week(p))
                Call Day_Toggle(days_of_week(d))
                Call Day_Toggle(days_of_week(h))
                
                If Range("FCI").Cells(1, 1).Value < Range("FCI_limit").Cells(1, 1).Value Then
                    
                    If Range("Profit").Cells(1, 1).Value > temp_max Then
                        temp_max = Range("Profit").Cells(1, 1).Value
                        temp_max_index1 = s
                        temp_max_index2 = p
                        temp_max_index3 = d
                        temp_max_index4 = h
                    End If
                End If
            End If
        Next s
        End If
    Next d
    End If
    Next h
    End If
    
Next p
'If a solution was not found then another try is made with a higher replenishment value
If temp_max = -9999 Then
    For p = 1 To 7
    If p <> y Then
    For h = 1 To 7
    Range("Selected_Days").ClearContents
    If h <> p And h <> y Then
    For d = 1 To 7
        
        If d <> y And d <> h And d <> p Then
        
        For s = 1 To 7

            If s <> y And s <> d And s <> p And s <> d Then
            
                Range("R_Amounts").Cells(s, 1).Value = Application.WorksheetFunction.Min(20000, Rep_Amount * 1.35 - (Rep_Amount * 1.35 Mod 20))
                Range("R_Amounts").Cells(y, 1).Value = Application.WorksheetFunction.Min(20000, Rep_Amount * 1.35 - (Rep_Amount * 1.35 Mod 20))
                Range("R_Amounts").Cells(p, 1).Value = Application.WorksheetFunction.Min(20000, Rep_Amount * 1.35 - (Rep_Amount * 1.35 Mod 20))
                Range("R_Amounts").Cells(d, 1).Value = Application.WorksheetFunction.Min(20000, Rep_Amount * 1.35 - (Rep_Amount * 1.35 Mod 20))
                Range("R_Amounts").Cells(h, 1).Value = Application.WorksheetFunction.Min(20000, Rep_Amount * 1.35 - (Rep_Amount * 1.35 Mod 20))
                Call Day_Toggle(days_of_week(y))
                Call Day_Toggle(days_of_week(s))
                Call Day_Toggle(days_of_week(p))
                Call Day_Toggle(days_of_week(d))
                Call Day_Toggle(days_of_week(h))
                
                If Range("FCI").Cells(1, 1).Value < Range("FCI_limit").Cells(1, 1).Value Then
                    
                    If Range("Profit").Cells(1, 1).Value > temp_max Then
                        temp_max = Range("Profit").Cells(1, 1).Value
                        temp_max_index1 = s
                        temp_max_index2 = p
                        temp_max_index3 = d
                        temp_max_index4 = h
                    End If
                End If
            End If
        Next s
        End If
    Next d
    End If
    Next h
    End If
    
Next p

  
        
        
        
End If
    'checks to make sure a feasible schedule has been found, adds a replenishment if not
    If temp_max = -9999 Then
        Num_Rep = 6
    Else
        
        Range("Selected_Days").ClearContents
        Call Day_Toggle(days_of_week(temp_max_index1))
        Call Day_Toggle(days_of_week(temp_max_index2))
        Call Day_Toggle(days_of_week(temp_max_index3))
        Call Day_Toggle(days_of_week(temp_max_index4))
        Call Day_Toggle(days_of_week(y))
    
    End If
End If
    
    
'Start of 6 replenishments case
If Num_of_Rep = 6 Then
Rep_Amount = Application.WorksheetFunction.Min(CInt(total_weekly / 5 - (total_weekly / 5 Mod 20)), Range("ATM_Cash_Limit").Cells(1, 1).Value)
Range("Selected_Days").ClearContents
For i = 1 To 7
    If i <> ZZ Then
        Range("R_Times").Cells(p, 1).Value = "4:00:00 AM"
        Range("R_Amounts").Cells(i, 1).Value = Rep_Amount
        Call Day_Toggle(days_of_week(i))
    End If
Next i
Range("Selected_Days").ClearContents
If Range("FCI").Cells(1, 1).Value > Range("FCI_limit").Cells(1, 1).Value Then
    For i = 1 To 7
        If i <> ZZ Then
        Range("R_Times").Cells(p, 1).Value = "4:00:00 AM"
        Range("R_Amounts").Cells(i, 1).Value = Application.WorksheetFunction.Min(20000, Rep_Amount * 1.5 - (Rep_Amount * 1.5 Mod 20))
        Call Day_Toggle(days_of_week(i))
        End If
    Next i
End If
    If Range("FCI").Cells(1, 1).Value < Range("FCI_limit").Cells(1, 1).Value Then
        Num_of_Rep = 7
    End If
End If

        
    
'Start of 7 replenishments case
If Num_of_Rep = 7 Then
Range("Selected_Days").ClearContents
For i = 1 To 7
    Range("R_Amounts").Cells(i, 1).Value = 20000
    Range("R_Times").Cells(i, 1).Value = "4:00:00 AM"
    Call Day_Toggle(days_of_week(i))
Next i
End If

    



End Sub


